In [1]:
import logging
import pandas as pd
import requests
import sqlite3
from datetime import date, datetime

In [2]:
log_format = "%(asctime)s::%(levelname)s::%(name)s::""%(filename)s::%(lineno)d::%(message)s"
logging.basicConfig(filename='./logs/get_coin_price.log', level='DEBUG', format=log_format)

In [3]:
def get_coin_price(ini = '01-01-2022', end = '31-03-2022'):
    """Takes two utc dates in the format dd-mm-yyyy and gets historic prices from the coingecko api, and returns them in a dataframe, grouped by date, requires pandas as pd, requests and datetime"""
    
    btc_start_date = int(datetime.strptime('28-04-2013', "%d-%m-%Y").timestamp())
    current_timestamp = int(datetime.now().timestamp())
    
    ini_timestamp = int(datetime.strptime(ini, "%d-%m-%Y").timestamp())
    end_timestamp = int(datetime.strptime(end, "%d-%m-%Y").timestamp())
    
    status = None
    
    if ini_timestamp < btc_start_date:
        ini_timestamp = btc_start_date
    
    if end_timestamp > current_timestamp:
        end_timestamp = current_timestamp

    try:
        coin_url = f'https://api.coingecko.com/api/v3/coins/bitcoin/market_chart/range?vs_currency=usd&from={ini_timestamp}&to={end_timestamp}&precision=2'
        request = requests.get(coin_url)
        coin_historic_prices_json = request.json()
    
    except requests.exceptions.RequestException as e:
        logging.info(f"Exception - {e} occured")
    
    finally:
        if request.status_code != 200:
            status = "FAIL"
            logging.info(f"get_coin_price : {status}")
        else:
            status = "PASS"
            logging.info(f"get_coin_price : {status}")
    
    return coin_historic_prices_json

In [4]:
def coin_price_formatter(coin_historic_prices_json):
    """Converts a json historic price to a dataframe and returns daily data"""
    
    coin_price_df = pd.DataFrame(coin_historic_prices_json['prices'], columns =['utc_epoch', 'usd_price'])
    
    coin_price_df['utc_epoch'] = pd.to_numeric(coin_price_df['utc_epoch'], errors='coerce')
    coin_price_df['usd_price'] = pd.to_numeric(coin_price_df['usd_price'], errors='coerce')
    coin_price_df.dropna(how='any', inplace=True)
    
    coin_price_df['utc_datetime'] = pd.to_datetime(coin_price_df['utc_epoch'], unit='ms').dt.date
    coin_price_df.drop(['utc_epoch'], axis = 1, inplace = True)
    
    coin_price_df = coin_price_df.groupby(['utc_datetime']).mean().reset_index()
    coin_price_df['usd_price'] = coin_price_df['usd_price'].round(2)
    
    logging.info(f"coin_price_formatter processed {len(coin_price_df)} records")
    coin_price_df.to_csv('./data/price_of_coins/coin_price.csv', index=False)
    
    return coin_price_df

In [5]:
get_coin_price.__doc__

'Takes two utc dates in the format dd-mm-yyyy and gets historic prices from the coingecko api, and returns them in a dataframe, grouped by date, requires pandas as pd, requests and datetime'

In [6]:
bitcoin_price_json = get_coin_price()

In [7]:
bitcoin_price = coin_price_formatter(bitcoin_price_json)

In [8]:
bitcoin_price

,utc_datetime,usd_price
0,2022-01-01,47309.21
1,2022-01-02,47317.14
2,2022-01-03,46891.53
3,2022-01-04,46498.01
4,2022-01-05,45971.16
...,...,...
85,2022-03-27,44854.32
86,2022-03-28,47226.49
87,2022-03-29,47497.49
88,2022-03-30,47230.60


In [9]:
conn = sqlite3.connect('./db/coingecko')
c = conn.cursor()

In [10]:
c.execute('CREATE TABLE IF NOT EXISTS bitcoin_price_hist (utc_datetime, usd_price)')
conn.commit()

In [11]:
bitcoin_price.to_sql('bitcoin_price_hist', conn, if_exists='replace')

90